In [ ]:
import boto3
import os
import pandas as pd
import swifter

In [ ]:
!pip install swifter

In [ ]:
!pip install boto3

In [ ]:
s3 = boto3.client("s3")

bucket = 'rec-gov-study'
file = 'data/df_merge_all_data_camping_unique_facil_start_end.csv'

date_cols = ['startdate', 'enddate', 'orderdate']

boto_object = s3.get_object(Bucket=bucket, Key=file)
df = pd.read_csv(boto_object['Body'], parse_dates=date_cols, nrows=300000)

df['enddate'] = pd.to_datetime(df['enddate'], errors='coerce')

In [ ]:
df.info()

In [ ]:
def reservations_likely_canceled(row):
    '''if there is another reservation made at a later date that has the same facility and 
    productuctid and is within the same reservation date block (between start and end) then 
    consider this one to be a likely canceled one.'''
    
    
    facil = row['facilityid']
    campsite = row['productid']
    sdate = row['startdate']
    edate = row['enddate']
    odate = row['orderdate']
    
    
    #select all rows where the start date is after and facil and campsite number are the same
    #(StartA <= EndB) and (EndA >= StartB)
    df_other_res = df[(df['facilityid']==facil) & 
                  (df['productid']==campsite) & 
                  (df['startdate']<=edate) &
                  (df['enddate']>=sdate) &
                  (df['orderdate']>odate)]
    
    if df_other_res.empty:
        return False
    else:
        return True

In [ ]:
%%time
df['cancelation_likely'] = df.swifter.apply(reservations_likely_canceled, axis=1)